In [ ]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 12.1 MB/s eta 0:00:00


In [ ]:
!playwright install


167.7 MiB [] 0% 11.1s167.7 MiB [] 0% 15.8s167.7 MiB [] 0% 11.0s167.7 MiB [] 0% 7.2s167.7 MiB [] 0% 7.4s167.7 MiB [] 1% 6.6s167.7 MiB [] 1% 5.7s167.7 MiB [] 2% 5.2s167.7 MiB [] 2% 4.7s167.7 MiB [] 3% 4.3s167.7 MiB [] 3% 4.1s167.7 MiB [] 4% 3.9s167.7 MiB [] 4% 3.8s167.7 MiB [] 5% 3.8s167.7 MiB [] 5% 4.0s167.7 MiB [] 6% 4.1s167.7 MiB [] 6% 4.0s167.7 MiB [] 7% 4.0s167.7 MiB [] 7% 4.1s167.7 MiB [] 7% 4.2s167.7 MiB [] 8% 4.0s167.7 MiB [] 8% 3.9s167.7 MiB [] 9% 4.0s167.7 MiB [] 9% 3.9s167.7 MiB [] 10% 3.8s167.7 MiB [] 11% 3.7s167.7 MiB [] 11% 3.6s167.7 MiB [] 12% 3.5s167.7 MiB [] 13% 3.3s167.7 MiB [] 14% 3.2s167.7 MiB [] 15% 3.1s167.7 MiB [] 15% 3.0s167.7 MiB [] 16% 3.0s167.7 MiB [] 17% 3.0s167.7 MiB [] 17% 2.9s167.7 MiB [] 18% 2.8s167.7 MiB [] 19% 2.7s167.7 MiB [] 20% 2.7s167.7 MiB [] 21% 2.7s167.7 MiB [] 21% 2.8s167.7 MiB [] 22% 2.7s167.7 MiB [] 23% 2.6s167.7 MiB [] 24% 2.5s167.7 MiB [] 26% 2.4s167.7 MiB [] 26% 2.3s167.7 MiB [] 28% 2.2s167.7 MiB [] 29% 2.1s167.7 MiB [] 30% 2.1s167.7 MiB [] 

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd

async def scrape_property(page, url):
    await page.goto(url, timeout=60000)
    await page.wait_for_timeout(5000)
    content = await page.content()
    soup = BeautifulSoup(content, "html.parser")

    # Title
    title_anchor = soup.select_one("h4.elementor-heading-title a")
    title = title_anchor.get_text(strip=True) if title_anchor else "N/A"

    # Sidebar Info
    sidebar_data = {}
    sidebar = soup.find("ul", class_="key-info-list")
    if sidebar:
        for li in sidebar.find_all("li"):
            key = li.find("span", class_="title").get_text(strip=True)
            val = li.find("span", class_="value").get_text(strip=True)
            sidebar_data[key] = val

    # Description
    desc_block = soup.select_one("div.elementor-widget-text-editor p")
    description = desc_block.get_text(strip=True) if desc_block else "N/A"

    # Availability
    availability_section = soup.select_one("div[data-id='f3ae1af']")
    availability_spans = availability_section.select("span.elementor-icon-list-text") if availability_section else []
    available_from = availability_spans[0].get_text(strip=True) if len(availability_spans) > 0 else "N/A"
    available_to = availability_spans[1].get_text(strip=True) if len(availability_spans) > 1 else "N/A"

    # Contact Name
    contact_spans = soup.select("span.elementor-heading-title.elementor-size-default")
    contact_name = "N/A"
    for i, span in enumerate(contact_spans):
        if "Name:" in span.get_text(strip=True):
            if i + 1 < len(contact_spans):
                contact_name = contact_spans[i + 1].get_text(strip=True)
            break

    # Contact Info
    all_text_spans = soup.select("span.elementor-icon-list-text")
    span_texts = [span.get_text(strip=True) for span in all_text_spans]
    mobile_number = span_texts[0] if len(span_texts) > 0 else "N/A"
    email = span_texts[2] if len(span_texts) > 2 else "N/A"

    # Room Info (max 2 rooms)
    room_details = soup.select("div.elementor-shortcode div.bd-group-field, div.elementor-shortcode div.sb-group-field")
    room_data = []

    for block in room_details[:2]:
        room = {
            "Room Type": "N/A",
            "Bed Available": "N/A",
            "Weekly Pay": "N/A",
            "Deposit": "N/A",
            "Monthly Pay": "N/A"
        }
        for p in block.find_all("p"):
            text = p.get_text(strip=True)
            if ":" in text:
                k, v = text.split(":", 1)
                room[k.strip()] = v.strip()
        room_data.append(room)

    room1 = room_data[0] if len(room_data) > 0 else {}
    room2 = room_data[1] if len(room_data) > 1 else {}

    # Room Facilities
    facilities = {}
    facility_section = soup.select_one("div.elementor-shortcode .property-suitable-checkboxes")
    if facility_section:
        for label in facility_section.select("label"):
            feature = label.get_text(strip=True)
            has_tick = label.select_one("svg[style*='green']")
            facilities[feature] = "Yes" if has_tick else "No"

    # Final structured data
    data = {
        "Title": title,
        "Description": description,
        "Available From": available_from,
        "Available To": available_to,
        "Mobile Number": mobile_number,
        "Contact Name": contact_name,
        "URL": url,
        "Email": email,
        **sidebar_data,

        # Room 1
        "Room1 Type": room1.get("Room Type", "N/A"),
        "Room1 Beds": room1.get("Bed Available", "N/A"),
        "Room1 Weekly Pay": room1.get("Weekly Pay", "N/A"),
        "Room1 Monthly Pay": room1.get("Monthly Pay", "N/A"),
        "Room1 Deposit": room1.get("Deposit", "N/A"),

        # Room 2
        "Room2 Type": room2.get("Room Type", "N/A"),
        "Room2 Beds": room2.get("Bed Available", "N/A"),
        "Room2 Weekly Pay": room2.get("Weekly Pay", "N/A"),
        "Room2 Monthly Pay": room2.get("Monthly Pay", "N/A"),
        "Room2 Deposit": room2.get("Deposit", "N/A"),
    }

    for facility, status in facilities.items():
        data[f"Facility - {facility}"] = status

    return data

async def scrape_multiple_properties(urls):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        all_data = []

        for url in urls:
            print(f"Scraping: {url}")
            try:
                data = await scrape_property(page, url)
                all_data.append(data)
            except Exception as e:
                print(f"❌ Failed to scrape {url}: {e}")

        await browser.close()
        df = pd.DataFrame(all_data)
        print("\n======= Final Combined DataFrame =======\n")
        display(df)
        df.to_csv("athlone2.csv", index=False)

# Example URLs (you can add more here)
urls = [
    "https://tussu.ie/accommodation-listin/curraghaleen-athlone-roscommon-n37a0k2/",
    "https://tussu.ie/accommodation-listin/52-corran-riada-athlone-roscommon-n37n1y6/",
    "https://tussu.ie/accommodation-listin/barrymore-kiltoom-athlone-westmeath-n37-y2y4/"
    ]

# Run batch scrape
await scrape_multiple_properties(urls)


Scraping: https://tussu.ie/accommodation-listin/curraghaleen-athlone-roscommon-n37a0k2/
Scraping: https://tussu.ie/accommodation-listin/52-corran-riada-athlone-roscommon-n37n1y6/
Scraping: https://tussu.ie/accommodation-listin/barrymore-kiltoom-athlone-westmeath-n37-y2y4/

======= Final Combined DataFrame =======



,Title,Description,Available From,Available To,Mobile Number,Contact Name,URL,Email,Room1 Type,Room1 Beds,...,Room2 Type,Room2 Beds,Room2 Weekly Pay,Room2 Monthly Pay,Room2 Deposit,Facility - Gas,Facility - Electricity,Facility - Heating,Facility - Bins,Facility - Broadband/Wifi
0,"curraghaleen, Athlone, Roscommon, N37A0K2",N/A,29/09/2025,19/12/2025,0863095492,Anthony,https://tussu.ie/accommodation-listin/curragha...,ahughes200@yahoo.co.uk,Double Bed,2,...,N/A,N/A,N/A,N/A,N/A,No,Yes,Yes,Yes,Yes
1,"52 Corran Riada, Athlone , Roscommon, N37N1Y6",Double Room with private bathroom.,23/06/2025,22/12/2025,0834863743,Kathleen,https://tussu.ie/accommodation-listin/52-corra...,Khynes2189@gmail.com,Double Bed,1,...,N/A,N/A,N/A,N/A,N/A,Yes,Yes,Yes,Yes,Yes
2,"Barrymore, Kiltoom , Athlone, Westmeath, N37 Y2Y4",Property Description –Available from Saturday ...,03/07/2025,05/09/2025,0871498371,Derek,https://tussu.ie/accommodation-listin/barrymor...,derekfkeenan@gmail.com,Double Bed,1,...,N/A,N/A,N/A,N/A,N/A,No,Yes,Yes,Yes,Yes
